## HDP-LDA Gibbs Sampling Notes



$$
\begin{array}{rl}
  H & \sim \text{Dirichlet}(\beta) \\
  G_0 \,|\, \gamma, H & \sim \text{DP}(\gamma, H) \\
  G_j \,|\, \alpha_0, G_0 & \sim \text{DP}(\alpha_0, G_0) \\
  \theta_{ji} \,|\, G_j & \sim G_j \\
  x_{ij} \,|\, \theta_{ji} & \sim \text{Categorical}(\theta_{ji}) \\
\end{array}
$$

* $H$ is Dirichlet distribution whose dimension is the size of the vocabulary, i.e. it is distribution over an uncountably-number of term distributions (topics). 
* $G_0$ is a distribution over a countably-infinite number of ategorical term distributions, i.e. topics.
* For each document $j$, $G_j$ is a is a distribution over a countably-infinite number of categorical term distributions, i.e. topics.
* $\theta_{ji}$ is a categorical distribution over terms, i.e. a topic.
* $x_{ij}$ is a term.

### Chinese Restaurant Franchise Approach

Instead of the above Dirichlet process model, we can think of an identical CRF model.

Each $\theta_{ji}$ is a customer in restaurant $j$. Each customer is sitting at a table, and each table has multiple customers. 

There is a global menu of $K$ dishes that the restaurants serve, $\phi_1,\ldots,\phi_K$. 

Some other definition:

* $\psi_{jt}$ is the dish served at table $t$ in restaurant $j$; i.e. each $\psi_{jt}$ corresponds to some $\phi_k$.
* $t_{ji}$ is the index of the $\psi_{jt}$ associated with $\theta_{ji}$. 
* $k_{jt}$ is the index of $\phi_k$ associated with $\psi_{jt}$.

_Customer $i$ in restaurant $j$ sits at table $t_{ji}$ while table $t$ in restaurant $j$ serves dish $k_{jt}$._

There are two arrays of count variables we will want to track:

* $n_{jtk}$ is the number of customers in restaurant $j$ at table $t$ eating dish $k$.
* $m_{jk}$ is the number of tables in restaurant $j$ serving dish $k$ (multiple tables may serve the same dish).

Supposed we want to know the distribution of $\theta_{ji}$ given $\theta_{j1},\ldots,\theta_{j,i-1},\alpha_0, G_0$. It is given by:

$$\theta_{ji}\,|\,\theta_{j1},\ldots,\theta_{j,i-1},\alpha_0, G_0 \sim \sum_{t=1}^{m_j}\frac{n_{jt\cdot}}{i-1+\alpha_0}\delta_{\psi_{jt}}+\frac{\alpha_0}{i-1+\alpha_0}G_0$$

This is a mixture. With some probability $\theta_{ji}$ will be the same dish (topic) $\psi_{jt}$ already assigned to another person, and with some probability, a new dish $\psi_{jm_j}$ (topic) is drawn from $G_0$ and $\theta_{jm}=\psi_{jm_j}$.

How do we draw from $G_0$. It is also a Dirichlet process where

$$\gamma_{jt} \,|\, \gamma_{11}, \gamma_{12}, \ldots, \gamma_{21}, \ldots, \gamma_{jt-1}, \gamma, H \sim \sum_{k=1}^K \frac{m_{\cdot k}}{m_{\cdot\cdot}+\gamma}+\frac{\gamma}{m_{\cdot\cdot}+\gamma}H$$

> To use these equations to obtain samples of $\theta_{ji}$, we proceed as follows. For each $j$ and $i$, first sample $\theta_ji$ using [the first distribution]. If a new sample from $G_0$ is needed, we use [the second distribution] to obtain a new sample $\psi_{jt}$ and set $\theta_{ji}=\psi_{jt}$.

To summarize:

$x_{ij}$ are observed data (words). We assume $x_{ij}\sim F(\theta_{ij})$. Further, we assume $\theta_{ji}$ is associated with table $t_{ji}$, that is $\theta_{ji}=\psi_{jt_{ji}}$. Further, we assume the topic for table $j$ is indexed by $k_{jt}$, i.e. $\psi_{jt}=\phi_{k_{jt}}$. Thus, if we know $t_{ji}$ (the table assignment for $x_{ij}$) and $k_{jt}$ (the dish assignment for table $t$) for all $i, j, t$, we could determine the remaining parameters $\phi$'s by sampling.

### HDP-LDA Updates

Teh's original HDP paper does not explain how to apply equation (30) (for $f_k^{x_{ji}}(x_{ji})$) to the LDA case. [A Blogger Formerly Known As Shuyo](https://shuyo.wordpress.com/2012/08/15/hdp-lda-updates/) briefly derives the update equation when $H$ is a Dirichlet distribution over terms distributions and $F$ is a multinomial distribution over terms. By definition, $h(\phi_k)=\frac{1}{Z}\prod_v[\phi_k]_v^{\beta-1}$ and $f(x_{ji}=v \,|\, \phi_k)=\phi_{kv}$.

For convenience, define $v=x_{ji}$ (word $i$ in document $j$), define $k=k_{jt_{ji}}$ (topic assignment for the table in document $j$ containing word $i$), and 

$$
    n_{kv}^{-ji}=\left|\left\{
        x_{mn} \,|\, k_{mt_{mn}}=k,\, x_{mn}=v,\, (m,n)\neq(j,i)
    \right\}\right|
$$

(the number of time the term $x_{ji}$, besides $x_{ji}$ itself, is generated by the same topic as was $x_{ji}$).

First look at the term (for fixed $k$):

$$
   \prod_{j'i'\neq ji, z_{j'i'=k}}
        f(x_{j'i'} \,|\, \phi_k)=    
    \prod_{j'}
    \prod_{i'\neq i, z_{j'i'}=k}
        [\phi_{k}]_{x_{j'i'}}
$$

are the inverse of normalizing coefficients for 

$[\phi_k]_v$ is the probability that term $v$ is generated by topic $k$. The double sums run over every word generated by topic $k$ in every document. Since $[\phi_{k}]_{x_{j'i'}}$ is fixed for a given word $w$, we could instead do a product over the each word of the vocabulary:


$$
\prod_{j'i'\neq ji, z_{j'i'=k}}
        f(x_{j'i'} \,|\, \phi_k)
    =\prod_{w\in\mathcal{V}}[\phi_k]_w^{n_{kw}^{-ji}}
$$


We use this early on in the big derivation below.

Also, note that 

$$
        \int
            \phi_{kv}^{n_{kv}^{-ji}+\beta}
            \prod_{w\neq v}
                \phi_{kw}^{n_{kw}^{-ji}+\beta-1}
        d\phi_k  \text{   and    }
                    \int
                \prod_w
                    \phi_{kw}^{n_{kw}^{-ji}+\beta-1}
            d\phi_k
$$

are the normalizing coefficients for Dirichlet distributions.

Equation (3) in Teh's paper is:

$$
\begin{array}{rl}
    f_k^{-x_{ji}}(x_{ji})
    &=\frac{
            \int
                f(x_{ji} \,|\, \phi_k)
                   \left[
                   \prod_{j'i'\neq ji, z_{j'i'}=k}
                        f(x_{j'i'} \,|\, \phi_k)
                   \right]
                        h(\phi_k)
                        d\phi_k
        }
        {
            \int
                \left[
                    \prod_{j'i'\neq ji, z_{j'i'}=k}
                    f(x_{j'i'}|\phi_k)
                \right]
                h(\phi_k)d\phi_k
        } \\
    &=\frac{
            \int
                f(x_{ji} \,|\, \phi_k)
                    \left[
                    \prod_{j'}
                    \prod_{i'\neq i, z_{j'i'}=k}
                        \phi_{kv} 
                    \right]\cdot
                        h(\phi_k)
                        d\phi_k
        }
        {
            \int
                \left[
                    \prod_{j'i'\neq ji, z_{j'i'}=k}
                    f(x_{j'i'}|\phi_k)
                \right]h(\phi_k)d\phi_k
        } \\
    &\propto\frac{
            \int
                \phi_{kv}
                \prod_w
                    \phi_{kw}^{n_{kw}^{-ji}}
                \prod_{w}
                    \phi_{kw}^{\beta-1}
            d\phi_k
        }{
            \int
                \prod_w
                    \phi_{kw}^{n_{kw}^{-ji}}
                \prod_w
                    \phi_{kw}^{\beta-1}
            d\phi_k
        }\\
    &=\frac{
            \int
                \phi_{kv}\cdot
                \phi_{kv}^{n_{kv}^{-ji}}\cdot
                \prod_{w\neq v}
                    \phi_{kw}^{n_{kw}^{-ji}}\cdot
                \phi_{kv}^{\beta-1}\cdot
                \prod_{w\neq v}
                    \phi_{kw}^{\beta-1}
            d\phi_k
        }{
            \int
                \prod_w
                    \phi_{kw}^{n_{kw}^{-ji}}
                \prod_w
                    \phi_{kw}^{\beta-1}
            d\phi_k
        }\\
    &=
        \int
            \phi_{kv}^{n_{kv}^{-ji}+\beta}
            \prod_{w\neq v}
                \phi_{kw}^{n_{kw}^{-ji}+\beta-1}
        d\phi_k
        \cdot
        \frac{
            1
        }{
            \int
                \prod_w
                    \phi_{kw}^{n_{kw}^{-ji}+\beta-1}
            d\phi_k
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kv}^{-ji}+1\right)
            \prod_{w\neq v} \Gamma\left(\beta+n_{kw}^{-ji}\right)
        }{
            \Gamma\left(
                \sum_{w\neq v}
                    \left[
                         \beta+n_{kw}^{-ji}
                    \right]+
                (\beta+n_{kv}^{-ji}+1)\right)
        }\cdot
        \frac{
            1
        }{
            \int\prod_w 
                \left(\phi_{kw}\right)^{n_{kw}^{-ji}+\beta-1}
            d\phi_k
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kv}^{-ji}+1\right)
            \prod_{w\neq v} \Gamma\left(\beta+n_{kw}^{-ji}\right)
        }{
            \Gamma\left(
                \sum_{w\in\mathcal{V}}
                    \left[
                         \beta+n_{kw}^{-ji}
                    \right]
                +1\right)
        }\cdot
        \frac{
            1
        }{
            \int\prod_w 
                \left(\phi_{kw}\right)^{n_{kw}^{-ji}+\beta-1}
            d\phi_k
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kv}^{-ji}+1\right)
            \prod_{w\neq v} \Gamma\left(\beta+n_{kw}^{-ji}\right)
        }{
            \Gamma\left(
                \sum_{w\in\mathcal{V}}
                    \left[
                         \beta+n_{kw}^{-ji}
                    \right]
                +1\right)
        }\cdot
        \frac{
            \Gamma\left(
            \sum_{w\in\mathcal{V}}
                \left[
                    \beta+n_{kw}^{-ji}
                \right]
            \right)
        }{
            \prod_{w}\Gamma\left(\beta+n_{kw}^{-ji}\right)
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kv}^{-ji}+1\right)
            \prod_{w\neq v} \Gamma\left(\beta+n_{kw}^{-ji}\right)
        }{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}+1\right)
        }\cdot
        \frac{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}\right)
        }{
            \prod_{w}\Gamma\left(\beta+n_{kw}^{-ji}\right)
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kv}^{-ji}+1\right)
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}\right)
        }{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}+1\right)
        }\cdot
        \frac{
            \prod_{w\neq v} \Gamma\left(\beta+n_{kw}^{-ji}\right)
        }{
            \prod_{w}\Gamma\left(\beta+n_{kw}^{-ji}\right)
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kn}^{-ji}+1\right)
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}\right)
        }{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji} + 1\right)
            \Gamma\left(\beta+n_{kv}^{-ji}\right)
        }\\
    &=\frac{
            \Gamma\left(\beta+n_{kn}^{-ji}+1\right)
        }{
            \Gamma\left(\beta+n_{kv}^{-ji}\right)
        }\cdot
        \frac{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji}\right)
        }{
            \Gamma\left(V\beta+n_{k\cdot}^{-ji} + 1\right)
        }\\
    &=\frac{\beta+n_{kv}^{-ji}}{V\beta+n_{k\cdot}^{-ji}}
\end{array}
$$